In [6]:
def generarPasos(u, x):
    res = 0
    if x < 0.5:
        res = u*x
    elif x >= 0.5:
        res = u*(1-x)
    return res

In [7]:
x = 0.1
for i in range(50):
    print(f"X{i+1}: {x:.4f}")
    x = generarPasos(1.5, x)

X1: 0.1000
X2: 0.1500
X3: 0.2250
X4: 0.3375
X5: 0.5063
X6: 0.7406
X7: 0.3891
X8: 0.5836
X9: 0.6246
X10: 0.5631
X11: 0.6554
X12: 0.5169
X13: 0.7246
X14: 0.4131
X15: 0.6197
X16: 0.5705
X17: 0.6443
X18: 0.5336
X19: 0.6996
X20: 0.4505
X21: 0.6758
X22: 0.4863
X23: 0.7295
X24: 0.4058
X25: 0.6087
X26: 0.5870
X27: 0.6195
X28: 0.5707
X29: 0.6439
X30: 0.5341
X31: 0.6989
X32: 0.4517
X33: 0.6775
X34: 0.4837
X35: 0.7256
X36: 0.4116
X37: 0.6175
X38: 0.5738
X39: 0.6393
X40: 0.5411
X41: 0.6884
X42: 0.4674
X43: 0.7011
X44: 0.4483
X45: 0.6724
X46: 0.4913
X47: 0.7370
X48: 0.3945
X49: 0.5917
X50: 0.6124


In [8]:
# ============================================
# INSTALACIÓN E IMPORTACIÓN DE LIBRERÍAS
# ============================================

# --- Import packages ---
try:
    import micropip
    await micropip.install(['ipywidgets', 'bqplot', 'plotly'])
except (ImportError, AttributeError, NameError):
    import sys
    import subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'ipywidgets', 'bqplot', 'plotly'])

# --- Test imports ---
try:
    import ipywidgets as widgets
    print("✓ ipywidgets importado correctamente")
except ImportError:
    print("✗ Error al importar ipywidgets")

try:
    import bqplot as bq
    print("✓ bqplot importado correctamente")
except ImportError:
    print("✗ Error al importar bqplot")

try:
    import plotly.graph_objects as go
    print("✓ plotly importado correctamente")
except ImportError:
    print("✗ Error al importar plotly")

# --- JavaScript patch for bqplot dragging ---
from IPython.display import display, Javascript
display(Javascript("""
// Patch for bqplot Scatter.enable_move in JupyterLite
(function() {
    if (typeof window.bqplotScatterPatchApplied !== 'undefined') return;
    
    function waitForBqplot() {
        if (typeof window.require === 'undefined') {
            setTimeout(waitForBqplot, 100);
            return;
        }
        
        require(['bqplot'], function(bqplot) {
            if (!bqplot || !bqplot.Scatter || !bqplot.Scatter.prototype) {
                console.warn('bqplot.Scatter no encontrado para parchear');
                return;
            }
            
            const originalInitialize = bqplot.Scatter.prototype.initialize;
            bqplot.Scatter.prototype.initialize = function(options) {
                const result = originalInitialize.apply(this, arguments);
                if (this.model.get('enable_move')) {
                    this._enableDrag();
                }
                return result;
            };
            
            if (!bqplot.Scatter.prototype._enableDrag) {
                bqplot.Scatter.prototype._enableDrag = function() {
                    if (!this.d3el || !this.d3el.selectAll) return;
                    
                    const that = this;
                    this.d3el.selectAll('.object_grp')
                        .style('cursor', 'move')
                        .call(d3.drag()
                            .on('drag', function(event, d) {
                                const scales = that.model.get('scales');
                                const xScale = scales.x;
                                const yScale = scales.y;
                                
                                if (xScale && yScale) {
                                    const newX = xScale.scale.invert(event.x);
                                    const newY = yScale.scale.invert(event.y);
                                    
                                    that.model.set('x', [newX]);
                                    that.model.set('y', [newY]);
                                    that.model.save_changes();
                                }
                            })
                        );
                };
            }
            
            window.bqplotScatterPatchApplied = true;
            console.log('✓ Parche de arrastre bqplot aplicado correctamente');
        }, function(err) {
            console.warn('No se pudo cargar bqplot para parchear:', err);
        });
    }
    
    if (document.readyState === 'loading') {
        document.addEventListener('DOMContentLoaded', waitForBqplot);
    } else {
        waitForBqplot();
    }
})();
"""))

print("\n🎉 ¡Todo listo! Ejecuta la siguiente celda para el widget interactivo.")



✓ ipywidgets importado correctamente
✓ bqplot importado correctamente
✓ plotly importado correctamente


<IPython.core.display.Javascript object>


🎉 ¡Todo listo! Ejecuta la siguiente celda para el widget interactivo.


In [19]:
from ipywidgets import interact
from plotly.subplots import make_subplots

In [10]:
def secuencia(a, u, num):
    x = a
    res = [[],[a]]
    for i in range(num):
        res[0].append(i)
    for i in range(num-1):
        x = generarPasos(u, x)
        res[1].append(x)
    return res
        

In [24]:
mu = 2
start = 0.9
steps = 50
def update(mu=2.0, start=0.9, steps=50):
    coords = secuencia(start, mu, steps)
    
    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Scatter(x=coords[0], y=coords[1]))


interact(update, mu=(0, 4, 0.1))

interactive(children=(FloatSlider(value=2.0, description='mu', max=4.0), FloatSlider(value=0.9, description='s…

<function __main__.update(mu=2.0, start=0.9, steps=50)>

In [52]:
print(secuencia(0.1, 0.5, 10))

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0.1, 0.05, 0.025, 0.0125, 0.0063, 0.0031, 0.0016, 0.0008, 0.0004, 0.0002]]
